# Far-field Beam Simulation: ML Training Sets
## Simons Observatory Large Aperture Telescope
### Grace E. Chesmore and Jeff McMahon - McMahonCosmologyGroup

We will simulate the far-field beam of the SO LAT using ray-tracing technique. We will simulate the SO LAT geometry in five scenarios, with a range of surface panel errors. We will then compute the power spectrum of the far-field beams, to inspect how surface panel errors impact the measurement constraints for SO's measurement of the CMB. 

## Aperture Field Ray Trace
In this section we compute the far-field beams at two receiver-feed (rx) positions, with an adjuster offset RMS of $35\mu m$ on the primary and secondary mirrors. First, we define the geometry of the telescope, including scan resolution receiver-feed position. 

In [ ]:
import numpy as np
import sys
sys.path.append('/home/chesmore/Desktop/Code/holosim_paper/package/holosim-ml')
import scipy
import optics_analyze as oa
import tele_geo as tg
import ap_field as af
import pan_fitting as pf
import pan_mod as pm
import tele_geo as tg
import far_field as ff
import ap_fitting as afit
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("ggplot")
font = {"family": "Times New Roman", "weight": "normal", "size": 18}
plt.rc("font", **font)
plt.rcParams["image.cmap"] = "magma"

adj_pos_m1, adj_pos_m2 = pm.get_single_vert_adj_positions()

save = 1

rx_x = np.array([0, 0])
rx_z = np.array([-600 * (3 / 2), 600 * (3 / 2)])
el = np.array([oa.el_offset(-600 * (3 / 2)), oa.el_offset(600 * (3 / 2))])
az = np.array([0, 0])

In [ ]:
iters = 1000
f, axes = plt.subplots(1, len(rx_x), figsize=(9, 2))

for ii in range(0, iters):

    if np.mod(ii + 1, 10) == 0:
        print("Building far-field " + str(ii + 1) + "/" + str(iters))
    adj_1 = np.random.randn(77 * 5) * 35
    adj_2 = np.random.randn(69 * 5) * 35
    tele_geo = tg.initialize_telescope_geometry()
    # Define panels on M1 and M2. Here you can define the
    # magnitude of the adjuster offsets on each mirror:
    pan_mod_m2 = pm.panel_model_from_adjuster_offsets(
        2, adj_2, 1 * (ii + 1), (save)
    )  # Panel Model on M2
    pan_mod_m1 = pm.panel_model_from_adjuster_offsets(
        1, adj_1, 1 * (ii + 1), (save)
    )  # Panel Model on M1
    # Define FOV of RX. In other words, define directions of
    # outgoing rays from the RX.
    th = np.linspace(-np.pi / 2 - 0.28, -np.pi / 2 + 0.28, tele_geo.N_scan)
    ph = np.linspace(np.pi / 2 - 0.28, np.pi / 2 + 0.28, tele_geo.N_scan)
    # Define the path of the rays from the RX to the aperture plane

    rx1 = np.array([rx_x[0], 209.09, rx_z[0]])
    # Measurement 1
    tele_geo.N_scan = 100
    tele_geo.lambda_ = 0.003
    # Set offsets of Receiver Feed (RX):
    tele_geo.rx_x = rx1[0]
    tele_geo.rx_y = rx1[1]
    tele_geo.rx_z = rx1[2]
    tele_geo.el0 += el[0]
    tele_geo.az0 += az[0]
    rxmirror = af.ray_mirror_pts(rx1, tele_geo, th, ph)
    out1 = af.aperature_fields_from_panel_model(
        pan_mod_m1, pan_mod_m2, rx1, tele_geo, th, ph, rxmirror
    )

    beam1 = ff.far_field_sim(out1, tele_geo, rx1)
    amp1 = 20 * np.log10(abs(beam1[2, :]) / np.max(abs(beam1[2, :])))
    np.savetxt(
        "/data/chesmore/sim_out/rx00-600/rx_" + str(rx1) + "_" + str(ii + 1) + ".txt",
        np.c_[
            np.real(beam1[0, :]),
            np.real(beam1[1, :]),
            np.real(beam1[2, :]),
            np.imag(beam1[2, :]),
        ],
    )

    tele_geo = tg.initialize_telescope_geometry()
    rx2 = np.array([rx_x[1], 209.09, rx_z[1]])
    tele_geo.rx_x = rx2[0]
    tele_geo.rx_y = rx2[1]
    tele_geo.rx_z = rx2[2]
    tele_geo.el0 += el[1]
    tele_geo.az0 += az[1]
    rxmirror = af.ray_mirror_pts(rx2, tele_geo, th, ph)
    out2 = af.aperature_fields_from_panel_model(
        pan_mod_m1, pan_mod_m2, rx2, tele_geo, th, ph, rxmirror
    )

    beam2 = ff.far_field_sim(out2, tele_geo, rx2)
    amp2 = 20 * np.log10(abs(beam2[2, :]) / np.max(abs(beam2[2, :])))
    np.savetxt(
        "/data/chesmore/sim_out/rx00600/rx_" + str(rx2) + "_" + str(ii + 1) + ".txt",
        np.c_[
            np.real(beam2[0, :]),
            np.real(beam2[1, :]),
            np.real(beam2[2, :]),
            np.imag(beam2[2, :]),
        ],
    )

    if ii == 0:

        axes[0].set_ylabel("el. [arcmin]")
        axes[0].set_xlabel(r"az. [arcmin]")
        axes[1].set_xlabel(r"az. [arcmin]")
        first = axes[0].scatter(
            np.real(beam1[0, :]) * (180 / np.pi) * 60,
            np.real(beam1[1, :]) * (180 / np.pi) * 60,
            c=amp1,
            vmin=-60,
        )
        first = axes[1].scatter(
            np.real(beam2[0, :]) * (180 / np.pi) * 60,
            np.real(beam2[1, :]) * (180 / np.pi) * 60,
            c=amp2,
            vmin=-60,
        )
        axes[0].axis("equal")
        axes[1].axis("equal")
        f.colorbar(first, ax=axes, label="dB")
        plt.show()
